In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path

In [ ]:
train_df_path = Path('/kaggle/input/nlp-getting-started/train.csv')
test_df_path = Path('/kaggle/input/nlp-getting-started/test.csv')
sample_submission_path = Path('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
train_df = pd.read_csv(train_df_path)
test_df = pd.read_csv(test_df_path)

In [ ]:
train_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.isnull().mean()*100
test_df.isnull().mean()*100

In [ ]:
train_df = train_df.drop(columns=['location','id'],errors='ignore')
test_df = test_df.drop(columns=['location','id'], errors='ignore')

In [ ]:
train_df.head()

In [ ]:
def fillnull(dataframe: pd.core.frame.DataFrame):
    dataframe = dataframe.fillna('NULL')
    return dataframe

In [ ]:
train_df = fillnull(train_df)
test_df = fillnull(test_df)

In [ ]:
#percentage of target labels
train_df['target'].value_counts(normalize=True) * 100

In [ ]:
y = (train_df['target']).to_numpy()

In [ ]:
X = train_df.drop(columns='target')

In [ ]:
X = X['keyword'] + ". " +  X['text']

In [ ]:
# X = (X.drop(columns=['keyword', 'text'], errors='ignore')).to_numpy()
X = (X.drop(columns=['keyword', 'text'], errors='ignore'))

In [ ]:
X_test = test_df['keyword'] +  ". " +  test_df['text']

In [ ]:
X_test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidfvect(data):
    vectorizer = TfidfVectorizer()
    vectorizer = vectorizer.fit(data)
    return vectorizer
vectorizer = tfidfvect(X)
X_vect = vectorizer.transform(X)
X_test_vect = vectorizer.transform(X_test)

In [ ]:
X_vect.shape, X_test_vect.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
_X_vect = vectorizer.fit_transform(X)

In [ ]:
_X_vect.shape

In [ ]:
#splitting data in train and validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_vect, y,test_size = 0.2, random_state=42, stratify=y )

In [ ]:
y_val.shape

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

In [ ]:
accuracy = logreg.score(X_val, y_val)
print("Accuracy: ", accuracy)

In [ ]:
#knn
from sklearn.neighbors import KNeighborsClassifier
acc = []
for n_neighbors in range(1,1000, 5):
    knn = KNeighborsClassifier(n_neighbors)
    knn.fit(X_train, y_train)
    accuracy = knn.score(X_val, y_val)
    acc.append(accuracy)
    print("Accuracy: ", accuracy, "n:", n_neighbors)

In [ ]:
max(acc)

In [ ]:
#knn
knn = KNeighborsClassifier(n_neighbors=82)
knn.fit(X_train, y_train)
accuracy = knn.score(X_val, y_val)
acc.append(accuracy)
print("Accuracy: ", accuracy, "n:", n_neighbors)

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500, max_depth=4, random_state=42)
rf.fit(X_train, y_train)
accuracy = rf.score(X_val, y_val)
print('Accuracy:', accuracy)

In [ ]:
preds = logreg.predict(X_test_vect)

In [ ]:
submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
submission.drop(columns='target', inplace = True)

In [ ]:
submission['target'] = preds

In [ ]:
submission.to_csv('submission_logreg.csv', index = False)

In [ ]:
len(submission.columns)